In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
import requests_cache
import json
import lxml.html as lx
from lxml import etree
from bs4 import BeautifulSoup
import time
from tqdm import tqdm

import plotly.express as px
import geopandas as gpd
import plotly.graph_objects as go
requests_cache.install_cache("fetch_fips")

In [2]:
url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'

In [3]:
result = requests.get(url)
counties = json.loads(result.text)

In [4]:
px.set_mapbox_access_token(open("../../tools&keys/mapbox.txt").read())

We will use this map to further visualize the network performance by region between years.

# scrape the zip to get the fips

In [6]:
df = pd.read_csv('../data/measurement_test_2021-2023/2021_1.csv')

In [4]:
df.head(3)

,zip,throughput
0,95134,26.151214
1,94539,27.242447
2,95112,39.113374


In [5]:
len(df)

100000

In [6]:
url = 'https://www.zipinfo.com/cgi-local/zipsrch.exe'

In [7]:
response = requests.get(url, params = {
        'cnty': 'cnty',
        'zip': 95616,
        'Go': 'Go'
})
response.raise_for_status()

In [8]:
html = lx.fromstring(response.text)

In [9]:
html.xpath('//table/tr[2]/td[last()]/text()')

['06113', 'latitude and longitude']

In [10]:
html.xpath('//table/tr[2]/td[last()]/text()')[0].strip()

'06113'

In [11]:
def fetch_fips(zip):
    url = 'https://www.zipinfo.com/cgi-local/zipsrch.exe'
    params = {'cnty': 'cnty', 'zip': zip, 'Go': 'Go'}
    response = requests.get(url, params)
    if response.status_code != 200:
        return None

    html = lx.fromstring(response.text)
    fips = html.xpath('//table/tr[2]/td[last()]/text()')[0].strip()
    return fips

### Since we only focus on california, the zip code is from 90000 to 96162, we want to form a mapping for further use to map zip code to fips code

In [12]:
zip_fips_mapping = {}

In [13]:
for zip in tqdm(range(90000, 96163)):
    fips = fetch_fips(zip)
    if fips:
        zip_fips_mapping[zip] = fips
    time.sleep(0.1)

100%|███████████████████████████████████████████████████████████| 6163/6163 [35:22<00:00,  2.90it/s]


In [14]:
zip_fips_mapping

{90000: 'latitude and longitude',
 90001: '06037',
 90002: '06037',
 90003: '06037',
 90004: '06037',
 90005: '06037',
 90006: '06037',
 90007: '06037',
 90008: '06037',
 90009: '06037',
 90010: '06037',
 90011: '06037',
 90012: '06037',
 90013: '06037',
 90014: '06037',
 90015: '06037',
 90016: '06037',
 90017: '06037',
 90018: '06037',
 90019: '06037',
 90020: '06037',
 90021: '06037',
 90022: '06037',
 90023: '06037',
 90024: '06037',
 90025: '06037',
 90026: '06037',
 90027: '06037',
 90028: '06037',
 90029: '06037',
 90030: '06037',
 90031: '06037',
 90032: '06037',
 90033: '06037',
 90034: '06037',
 90035: '06037',
 90036: '06037',
 90037: '06037',
 90038: '06037',
 90039: '06037',
 90040: '06037',
 90041: '06037',
 90042: '06037',
 90043: '06037',
 90044: '06037',
 90045: '06037',
 90046: '06037',
 90047: '06037',
 90048: '06037',
 90049: '06037',
 90050: '06037',
 90051: '06037',
 90052: '06037',
 90053: '06037',
 90054: '06037',
 90055: '06037',
 90056: '06037',
 90057: '06037

In [20]:
a = 5
for k,v in zip_fips_mapping.items():
    print(k, " ", v, " ", len(v))
    a = a - 1
    if a == 0:
        break

90000   latitude and longitude   22
90001   06037   5
90002   06037   5
90003   06037   5
90004   06037   5


In [21]:
zip_fips_mapping = {k: v for k,v in zip_fips_mapping.items() if len(v) == 5}
zip_fips_mapping

{90001: '06037',
 90002: '06037',
 90003: '06037',
 90004: '06037',
 90005: '06037',
 90006: '06037',
 90007: '06037',
 90008: '06037',
 90009: '06037',
 90010: '06037',
 90011: '06037',
 90012: '06037',
 90013: '06037',
 90014: '06037',
 90015: '06037',
 90016: '06037',
 90017: '06037',
 90018: '06037',
 90019: '06037',
 90020: '06037',
 90021: '06037',
 90022: '06037',
 90023: '06037',
 90024: '06037',
 90025: '06037',
 90026: '06037',
 90027: '06037',
 90028: '06037',
 90029: '06037',
 90030: '06037',
 90031: '06037',
 90032: '06037',
 90033: '06037',
 90034: '06037',
 90035: '06037',
 90036: '06037',
 90037: '06037',
 90038: '06037',
 90039: '06037',
 90040: '06037',
 90041: '06037',
 90042: '06037',
 90043: '06037',
 90044: '06037',
 90045: '06037',
 90046: '06037',
 90047: '06037',
 90048: '06037',
 90049: '06037',
 90050: '06037',
 90051: '06037',
 90052: '06037',
 90053: '06037',
 90054: '06037',
 90055: '06037',
 90056: '06037',
 90057: '06037',
 90058: '06037',
 90059: '06037

In [71]:
import csv
output_file_path = 'zip_fips_mapping.csv'
with open(output_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ZIP', 'FIPS'])
    for zip, fips in zip_fips_mapping.items():
        writer.writerow([zip, fips])

# Visualize the 2021 throughput data across California

## First half

In [7]:
df = pd.read_csv('../data/measurement_test_2021-2023/2021_1.csv')
df.head(3)

,zip,throughput
0,95134,26.151214
1,94539,27.242447
2,95112,39.113374


In [8]:
mapping_df = pd.read_csv('../../tools&keys/zip_fips_mapping.csv', dtype={'FIPS': str})
mapping_df.head(3)

,ZIP,FIPS
0,90001,06037
1,90002,06037
2,90003,06037


In [9]:
zip_to_fips_series = mapping_df.set_index('ZIP')['FIPS']

In [10]:
df['fips'] = df['zip'].map(zip_to_fips_series)
df = df.drop('zip', axis=1)

In [11]:
df_throughput_by_fips = df.groupby('fips')['throughput'].mean().reset_index()

In [12]:
df_throughput_by_fips.head(3)

,fips,throughput
0,06001,56.191779
1,06005,70.934416
2,06007,75.370361


In [ ]:
fig = px.choropleth_mapbox(df_throughput_by_fips, geojson=counties, locations='fips', color='throughput',
                           color_continuous_scale="Inferno",
                           range_color=(20, 80),
                           mapbox_style="basic",
                           zoom=4, center = {"lat": 38, "lon": -121},
                           opacity=0.5,
                           labels={'trp':'average throughput'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
# fig.write_html('../graphs/2021_Jan_network.html')

# Set up a slider to see it on a timeline

In [51]:
df_2021_1 = df_throughput_by_fips.copy()
df_2021_1['period'] = '2021_1'
df_2021_1.head(3)

,fips,throughput,period
0,06001,56.191779,2021_1
1,06005,70.934416,2021_1
2,06007,75.370361,2021_1


In [52]:
df_combined = pd.DataFrame()
for period in ['2021_1', '2021_2', '2022_1', '2022_2', '2023_1', '2023_2']:
    df_sub = pd.read_csv(f'../data/measurement_test_2021-2023/{period}.csv')
    df_sub['fips'] = df_sub['zip'].map(zip_to_fips_series)
    df_sub = df_sub.drop('zip', axis=1)
    df_sub = df_sub.groupby('fips')['throughput'].mean().reset_index()
    df_sub['period'] = period
    df_combined = pd.concat([df_combined, df_sub])

In [53]:
df_combined

,fips,throughput,period
0,06001,56.191779,2021_1
1,06005,70.934416,2021_1
2,06007,75.370361,2021_1
3,06009,59.421754,2021_1
4,06011,53.427707,2021_1
...,...,...,...
51,06107,74.832271,2023_2
52,06109,55.466826,2023_2
53,06111,77.614473,2023_2
54,06113,74.777702,2023_2


In [ ]:
fig = px.choropleth_mapbox(df_combined, 
                           geojson=counties, 
                           locations='fips', 
                           color='throughput',
                           color_continuous_scale="Inferno",
                           range_color=(20, 80),
                           mapbox_style="carto-positron",
                           zoom=4, 
                           center={"lat": 38, "lon": -121},
                           opacity=0.5,
                           labels={'throughput':'Average Throughput'},
                           animation_frame='period'  # This is where the half-year period column is used
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.layout.pop("updatemenus")
fig.show()

In [55]:
fig.write_html('../graphs/2021-2023 slider.html')